# Device

CODE IN DEVELOPMENT

In [ ]:
import socket

from util import SERVER, SIZE_K, SIZE_N, decrypt, encrypt, hash, generate_n

def device():
    kdc_sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    kdc_sock.connect((SERVER["alias"], SERVER["port"] + 1))
    k_md = kdc_sock.recv(SIZE_K)
    print(f"device received AES key (k_md) from KDC: {k_md.hex()}")

    device_sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    device_sock.bind((SERVER["alias"], SERVER["port"] + 3))
    device_sock.listen(1)
    print("device listening on {}:{}".format(SERVER["alias"], SERVER["port"] + 3))

    conn, addr = device_sock.accept()
    print("connected to server at {}".format(addr))

    received_data = conn.recv(1024)
    ciphertext = received_data[:len(received_data) - len(auth_code)]
    auth_code = received_data[len(received_data) - len(auth_code):]
    nonce = auth_code[:SIZE_N]

    message = decrypt(ciphertext, k_md, nonce)
    if hash(message, k_md, nonce) == auth_code[SIZE_N:]:
        print(f"received message from server: {message.decode()}")
    else:
        print("authentication failed. L.")

    message = b"bruh"
    nonce = generate_n()
    ciphertext = encrypt(message, k_md, nonce)
    auth_code = hash(message, k_md, nonce)

    conn.send(ciphertext + auth_code)

    print("message sent to server")

if __name__ == "__main__":
    device()